In [72]:
import os
import tarfile

notebook_path = os.getcwd()
data_path = os.path.join(notebook_path, "download/training_set.tar")


def parse_customer_data(chunk):
    """
    Helper function to turn the customer data into usable data.
    Each line in our chunk corresponds to a rating from a customer
    and its date in the following format:
    CustomerID, Rating, Date
    
    returns:
        List of tuples containing the two elements 
        (CustomerID: int, Raring: int)
    """
    
    dat = []
    for cust_rating in chunk.split("\\n"):
        rating_data = cust_rating.split(",")
        if len(rating_data) == 3:
            
            customer_id, rating = map(int, rating_data[:2])
            
            print(cust_rating, customer_id, rating)
            dat.append((customer_id, rating))
            
    return dat

with tarfile.open(data_path) as tf:

    for entry in tf:
        fileobj = tf.extractfile(entry)
        
        if fileobj is not None:
            data = str(fileobj.read())
            movie_id, customer_data = data.split(":")
            
            movie_id = int(movie_id[2:])
            customer_data = parse_customer_data(customer_data)
            
            print(movie_id, customer_data)
            
            
            
            break

2071969,4,2005-07-07 2071969 4
1395430,3,2005-07-10 1395430 3
716091,4,2005-07-18 716091 4
2329101,5,2005-07-20 2329101 5
1017542,4,2005-08-01 1017542 4
2046595,2,2005-08-14 2046595 2
765331,4,2005-08-17 765331 4
464095,5,2005-08-25 464095 5
1242432,5,2005-08-25 1242432 5
779760,5,2005-08-25 779760 5
504195,4,2005-08-26 504195 4
2560405,5,2005-09-05 2560405 5
146851,4,2005-04-04 146851 4
1109948,3,2005-05-03 1109948 3
2439005,5,2005-05-04 2439005 5
1990901,5,2005-05-05 1990901 5
2093097,4,2005-05-09 2093097 4
1401310,4,2005-05-12 1401310 4
364518,4,2005-05-17 364518 4
574582,4,2005-05-20 574582 4
1907667,4,2005-05-23 1907667 4
1109700,4,2005-05-24 1109700 4
2581970,3,2005-05-24 2581970 3
2036099,5,2005-05-24 2036099 5
925220,5,2005-05-27 925220 5
1917746,1,2005-06-06 1917746 1
1091832,4,2005-06-07 1091832 4
146200,5,2005-06-20 146200 5
938385,5,2005-06-22 938385 5
876388,3,2005-06-26 876388 3
1463145,4,2005-06-27 1463145 4
1850680,1,2005-09-17 1850680 1
88271,5,2005-09-24 88271 5
16320

1436160,4,2005-07-14 1436160 4
1793459,5,2005-07-20 1793459 5
389966,4,2005-07-23 389966 4
415672,5,2005-07-29 415672 5
134378,4,2005-08-22 134378 4
1575462,5,2005-08-30 1575462 5
835847,5,2005-05-07 835847 5
1314869,1,2005-05-17 1314869 1
1219818,4,2005-05-18 1219818 4
266172,4,2005-05-25 266172 4
96641,3,2005-05-27 96641 3
307997,3,2005-06-07 307997 3
2181770,3,2005-06-08 2181770 3
1618010,5,2005-06-10 1618010 5
2148888,3,2005-06-16 2148888 3
2382025,4,2005-06-20 2382025 4
2157486,5,2005-06-27 2157486 5
1792741,4,2005-06-29 1792741 4
1930587,4,2005-09-02 1930587 4
1913817,4,2005-09-21 1913817 4
1947371,5,2005-09-28 1947371 5
192541,4,2005-10-10 192541 4
2414164,4,2005-10-21 2414164 4
1778441,5,2005-10-27 1778441 5
793875,4,2005-11-09 793875 4
2510037,2,2005-11-21 2510037 2
1583129,4,2005-12-29 1583129 4
308719,5,2005-07-06 308719 5
1979699,4,2005-07-15 1979699 4
1539430,4,2005-08-04 1539430 4
2543952,5,2005-08-22 2543952 5
1097536,5,2005-08-30 1097536 5
2578689,4,2005-03-24 2578689 4

1748059,4,2005-10-25 1748059 4
2125913,3,2005-11-21 2125913 3
2037001,5,2005-12-04 2037001 5
2499661,2,2005-07-18 2499661 2
339547,5,2005-08-19 339547 5
155620,4,2005-04-19 155620 4
13407,5,2005-04-25 13407 5
2495395,3,2005-04-30 2495395 3
1999176,4,2005-05-06 1999176 4
128389,4,2005-05-09 128389 4
2600868,5,2005-05-09 2600868 5
2020775,2,2005-05-10 2020775 2
1100528,5,2005-05-12 1100528 5
2217779,4,2005-05-21 2217779 4
1796464,4,2005-05-27 1796464 4
1517590,3,2005-05-27 1517590 3
309713,5,2005-05-30 309713 5
872784,4,2005-06-18 872784 4
637190,3,2005-06-25 637190 3
286665,5,2005-07-01 286665 5
998165,4,2005-09-15 998165 4
1798748,5,2005-09-23 1798748 5
2197203,4,2005-09-28 2197203 4
1193242,5,2005-10-04 1193242 5
1419909,1,2005-10-20 1419909 1
2132380,3,2005-10-24 2132380 3
2640999,1,2005-10-24 2640999 1
2017253,5,2005-10-31 2017253 5
2096987,4,2005-11-07 2096987 4
989890,4,2005-11-22 989890 4
1792392,4,2005-12-06 1792392 4
1736397,5,2005-12-25 1736397 5
1551367,4,2005-07-05 1551367 4

973313,4,2005-06-27 973313 4
2052390,4,2005-10-04 2052390 4
2604442,4,2005-10-07 2604442 4
1064093,3,2005-10-11 1064093 3
654518,3,2005-10-17 654518 3
2188580,5,2005-11-02 2188580 5
2306618,4,2005-11-08 2306618 4
878489,3,2005-12-24 878489 3
2623119,4,2005-07-11 2623119 4
2538611,4,2005-08-02 2538611 4
2040906,4,2005-08-03 2040906 4
33365,5,2005-08-03 33365 5
2083582,4,2005-08-03 2083582 4
2286785,5,2005-08-07 2286785 5
2203225,3,2005-08-16 2203225 3
368755,5,2005-08-25 368755 5
1331518,5,2005-05-04 1331518 5
2508234,4,2005-05-17 2508234 4
1132764,5,2005-05-19 1132764 5
1279851,3,2005-05-20 1279851 3
945621,5,2005-05-21 945621 5
1387145,5,2005-05-31 1387145 5
1684749,5,2005-06-01 1684749 5
2186799,4,2005-06-06 2186799 4
382633,4,2005-06-07 382633 4
1807427,5,2005-06-07 1807427 5
1463219,4,2005-06-14 1463219 4
373103,4,2005-06-29 373103 4
1075075,5,2005-10-08 1075075 5
900992,4,2005-10-10 900992 4
231329,5,2005-10-11 231329 5
1936497,4,2005-10-11 1936497 4
2139894,4,2005-07-10 2139894 4

154908,2,2005-07-23 154908 2
2605894,4,2005-07-26 2605894 4
87160,5,2005-08-01 87160 5
2599813,4,2005-08-18 2599813 4
1896786,4,2005-08-23 1896786 4
2535971,5,2005-08-26 2535971 5
467596,5,2005-03-21 467596 5
2035310,4,2005-04-11 2035310 4
2306848,3,2005-04-22 2306848 3
1870227,3,2005-05-01 1870227 3
831151,5,2005-05-18 831151 5
2564438,5,2005-05-22 2564438 5
896080,3,2005-05-22 896080 3
1048629,5,2005-05-25 1048629 5
110039,4,2005-05-25 110039 4
576723,4,2005-05-28 576723 4
578456,4,2005-05-29 578456 4
513930,4,2005-06-01 513930 4
42423,5,2005-06-13 42423 5
2081866,5,2005-06-14 2081866 5
2551046,4,2005-06-25 2551046 4
458009,4,2005-06-25 458009 4
995939,3,2005-06-27 995939 3
2036973,5,2005-06-29 2036973 5
164103,3,2005-09-14 164103 3
855377,5,2005-09-23 855377 5
2247406,3,2005-10-24 2247406 3
1891028,4,2005-11-02 1891028 4
2544518,5,2005-11-10 2544518 5
166317,4,2005-11-25 166317 4
2035261,5,2005-12-30 2035261 5
2517609,3,2005-07-06 2517609 3
2228441,3,2005-07-13 2228441 3
424818,4,20

330599,3,2005-07-05 330599 3
2352221,4,2005-09-12 2352221 4
2033181,5,2005-09-27 2033181 5
678922,5,2005-10-03 678922 5
1939632,3,2005-10-05 1939632 3
1181784,4,2005-10-16 1181784 4
1716311,4,2005-10-18 1716311 4
724896,4,2005-10-21 724896 4
1710658,3,2005-11-07 1710658 3
2503760,3,2005-12-04 2503760 3
2485060,4,2005-12-29 2485060 4
1863423,5,2005-07-09 1863423 5
2127784,5,2005-07-09 2127784 5
2337729,4,2005-07-11 2337729 4
2115514,4,2005-07-22 2115514 4
1403217,2,2005-07-25 1403217 2
726727,5,2005-08-02 726727 5
1982898,4,2005-08-09 1982898 4
1187765,3,2005-08-10 1187765 3
513664,4,2005-08-13 513664 4
120368,4,2005-02-27 120368 4
409309,3,2005-03-15 409309 3
2371144,5,2005-03-22 2371144 5
942758,5,2005-05-05 942758 5
58683,5,2005-05-12 58683 5
1851609,4,2005-05-16 1851609 4
2135817,5,2005-05-18 2135817 5
2276790,4,2005-05-19 2276790 4
881588,4,2005-05-28 881588 4
2401785,3,2005-06-06 2401785 3
2328825,4,2005-06-13 2328825 4
209486,4,2005-06-22 209486 4
1021397,3,2005-06-22 1021397 3
1

1459910,3,2005-05-19 1459910 3
322895,3,2005-05-27 322895 3
262545,4,2005-06-07 262545 4
2350141,4,2005-06-08 2350141 4
397929,3,2005-06-13 397929 3
1808694,4,2005-06-13 1808694 4
2611773,5,2005-06-28 2611773 5
166965,5,2005-09-14 166965 5
2526584,2,2005-09-25 2526584 2
2498939,5,2005-10-22 2498939 5
2342898,4,2005-10-31 2342898 4
575399,5,2005-11-01 575399 5
2073795,4,2005-11-08 2073795 4
1273224,5,2005-11-10 1273224 5
1743315,3,2005-11-12 1743315 3
1191821,5,2005-11-15 1191821 5
1734079,4,2005-12-20 1734079 4
1483814,3,2005-12-22 1483814 3
752642,4,2005-03-15 752642 4
330798,5,2005-08-18 330798 5
964001,4,2005-09-07 964001 4
2339869,4,2005-04-21 2339869 4
128040,5,2005-05-11 128040 5
2348266,5,2005-05-21 2348266 5
576019,3,2005-05-29 576019 3
577616,5,2005-05-31 577616 5
470779,5,2005-06-14 470779 5
1694440,4,2005-06-22 1694440 4
2033859,4,2005-09-16 2033859 4
339252,3,2005-10-27 339252 3
1580032,5,2005-11-04 1580032 5
1638987,5,2005-07-05 1638987 5
1188440,4,2005-07-11 1188440 4
172

2016801,3,2005-07-21 2016801 3
1006608,5,2005-07-22 1006608 5
1895828,5,2005-07-27 1895828 5
607897,4,2005-08-02 607897 4
1029163,4,2005-04-14 1029163 4
2361318,4,2005-05-07 2361318 4
612107,4,2005-05-18 612107 4
1556884,4,2005-05-20 1556884 4
803605,4,2005-05-22 803605 4
1817712,3,2005-05-24 1817712 3
671319,3,2005-05-25 671319 3
2289469,5,2005-06-04 2289469 5
1114324,1,2005-06-08 1114324 1
2084107,4,2005-06-14 2084107 4
574814,5,2005-06-16 574814 5
2365983,1,2005-06-17 2365983 1
1309057,5,2005-06-21 1309057 5
1206462,3,2005-06-29 1206462 3
2446913,4,2005-09-25 2446913 4
1273359,4,2005-09-26 1273359 4
1972191,4,2005-10-03 1972191 4
1334853,4,2005-10-06 1334853 4
1357107,5,2005-10-13 1357107 5
307920,3,2005-11-03 307920 3
1997815,4,2005-11-23 1997815 4
1087407,5,2005-11-25 1087407 5
301665,3,2005-12-03 301665 3
1759147,4,2005-12-09 1759147 4
35741,4,2005-07-05 35741 4
193294,3,2005-07-06 193294 3
2120912,4,2005-07-07 2120912 4
419030,4,2005-07-13 419030 4
240416,4,2005-07-15 240416 4
1

1314645,3,2005-08-05 1314645 3
1417314,4,2005-08-23 1417314 4
1370511,4,2005-08-28 1370511 4
2433995,3,2005-04-05 2433995 3
356403,5,2005-05-04 356403 5
2143500,5,2005-05-18 2143500 5
565161,2,2005-05-20 565161 2
692235,4,2005-05-23 692235 4
1049096,4,2005-05-25 1049096 4
1404078,4,2005-06-01 1404078 4
1871973,3,2005-06-14 1871973 3
1507727,5,2005-06-14 1507727 5
578690,4,2005-06-28 578690 4
538723,5,2005-09-16 538723 5
734347,4,2005-09-20 734347 4
570410,3,2005-10-05 570410 3
1272324,2,2005-11-09 1272324 2
2186650,4,2005-11-10 2186650 4
545234,4,2005-11-14 545234 4
1451860,5,2005-12-20 1451860 5
651589,4,2005-12-26 651589 4
1795543,1,2005-12-27 1795543 1
2261934,4,2005-07-13 2261934 4
1023470,5,2005-07-13 1023470 5
1797765,3,2005-07-17 1797765 3
2342120,3,2005-07-23 2342120 3
64464,4,2005-07-25 64464 4
1198356,4,2005-08-03 1198356 4
821568,3,2005-08-28 821568 3
1382816,3,2005-08-29 1382816 3
637990,5,2005-09-06 637990 5
1216880,4,2005-03-14 1216880 4
632138,5,2005-04-19 632138 5
19327

In [21]:
import numpy as np

def prox_gg(x):
    print("in gg", x.shape)
    u, s, v = np.linalg.svd(x, full_matrices=False)
    s_gamma = np.maximum(s - gamma, 0)
    return u @ s_gamma @ v


def prox_gf(x, A):
    return x + A.T @ (b - A @ x)
    
    
def DRS(A, b, gamma, maxiter):
    #print("shape: ",s_gamma.shape, s.shape)
    #print(s_gamma)
    z = np.zeros(A.shape[0])
    
    for _  in range(maxiter):
        z = z + prox_gg(2*prox_gf(z, A) - z) - prox_gf(z, A)
        
    return z
b = 2* np.ones(10)
A = np.ones((10,10))
DRS(A, b, 0.1, 1)

in gg (10,)


LinAlgError: 1-dimensional array given. Array must be at least two-dimensional